In [1]:
%pip install flask pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
data = '''
Preference Red_Wine White_Wine Recommendation
Red Light-Bodied None Pinot-Noir
Red Full-Bodied None Shiraz/Zinfandel
White None Dry Sauvignon-Blanc
White None None Sweet-Gewurztraminer
Red-Fruity None None Pinot-Noir
Red-Earthy None None Chianti
White-Crisp None None Sauvignon-Blanc
White-Creamy None None Chardonnay
Red-Spicy None None Shiraz/Zinfandel
Red-Rich None None Cabernet-Sauvignon
White-Floral None None Gewurztraminer
White-Citrus None None Riesling
Red None None Pinot-Noir
Red None None Chianti
White None None Sauvignon-Blanc
White None None Chardonnay
Red None None Shiraz/Zinfandel
Red None None Cabernet-Sauvignon
White None None Gewurztraminer
White None None Riesling
Red-Fruity Light-Bodied None Pinot-Noir
Red-Fruity Full-Bodied None Shiraz/Zinfandel
Red-Earthy Light-Bodied None Pinot-Noir
Red-Earthy Full-Bodied None Cabernet-Sauvignon
White-Crisp Dry None Sauvignon-Blanc
White-Crisp Sweet None Pinot-Noir
White-Creamy Dry None Pinot-Noir
White-Creamy Sweet None Chardonnay
Red-Spicy Light-Bodied None Pinot-Noir
Red-Spicy Full-Bodied None Shiraz/Zinfandel
Red-Rich Light-Bodied None Pinot-Noir
Red-Rich Full-Bodied None Cabernet-Sauvignon
White-Floral Dry None Pinot-Noir
White-Floral Sweet None Gewurztraminer
White-Citrus Dry None Sauvignon-Blanc
White-Citrus Sweet None Riesling
Red-Fruity None Sweet Pinot-Noir
Red-Fruity None Dry Pinot-Noir
Red-Earthy None Sweet-Chianti
Red-Earthy None Dry Pinot-Noir
'''

#Here import the data and all needed libraries 

import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import LabelEncoder
from flask import Flask, request, jsonify
from io import StringIO
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint


In [3]:
# Parse data with string io to emulate file parsing
dataFrame = pd.read_csv(StringIO(data), sep="\s+")
dataFrame = dataFrame.fillna("none")
print(dataFrame)

      Preference      Red_Wine     White_Wine        Recommendation
0            Red  Light-Bodied           none            Pinot-Noir
1            Red   Full-Bodied           none      Shiraz/Zinfandel
2          White          none            Dry       Sauvignon-Blanc
3          White          none           none  Sweet-Gewurztraminer
4     Red-Fruity          none           none            Pinot-Noir
5     Red-Earthy          none           none               Chianti
6    White-Crisp          none           none       Sauvignon-Blanc
7   White-Creamy          none           none            Chardonnay
8      Red-Spicy          none           none      Shiraz/Zinfandel
9       Red-Rich          none           none    Cabernet-Sauvignon
10  White-Floral          none           none        Gewurztraminer
11  White-Citrus          none           none              Riesling
12           Red          none           none            Pinot-Noir
13           Red          none           none   

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_10312\808151226.py:2: SyntaxWarning: invalid escape sequence '\s'
  dataFrame = pd.read_csv(StringIO(data), sep="\s+")


In [4]:
#Encode our data
le_preference =  LabelEncoder()
le_red_wine = LabelEncoder()
le_white_wine = LabelEncoder()
le_recommendation = LabelEncoder()

dataFrame['preference_n'] = le_preference.fit_transform(dataFrame['Preference'])
dataFrame['red_wine_n'] = le_red_wine.fit_transform(dataFrame['Red_Wine'])
dataFrame['white_wine_n'] = le_white_wine.fit_transform(dataFrame['White_Wine'])
dataFrame['recommendation_n'] = le_recommendation.fit_transform(dataFrame['Recommendation'])

dataFrame.head()

,Preference,Red_Wine,White_Wine,Recommendation,preference_n,red_wine_n,white_wine_n,recommendation_n
0,Red,Light-Bodied,none,Pinot-Noir,0,2,3,4
1,Red,Full-Bodied,none,Shiraz/Zinfandel,0,1,3,7
2,White,none,Dry,Sauvignon-Blanc,5,4,0,6
3,White,none,none,Sweet-Gewurztraminer,5,4,3,8
4,Red-Fruity,none,none,Pinot-Noir,2,4,3,4


In [5]:
# Make another dataframe with encoded texts only
dataframe_n = dataFrame.drop(['Preference', 'Red_Wine', 'White_Wine', 'Recommendation'], axis='columns')
dataframe_n.head()

,preference_n,red_wine_n,white_wine_n,recommendation_n
0,0,2,3,4
1,0,1,3,7
2,5,4,0,6
3,5,4,3,8
4,2,4,3,4


In [6]:
#split into features and target
input = dataframe_n.drop("recommendation_n", axis="columns")
target = dataframe_n['recommendation_n']

#train model 
x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.3)
randomModel = RandomForestClassifier(n_estimators=451)
randomModel.fit(x_train, y_train)

y_pred = randomModel.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.3333333333333333


In [7]:
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(x_train, y_train)

# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best hyperparameters: {'max_depth': 14, 'n_estimators': 179}


In [8]:
app = Flask(__name__)

@app.route('/random_forest', methods=['POST'])
def recommend():
    preferences = request.json    
    input_data = pd.DataFrame([preferences])

    

    input_data['preference_n'] = le_preference.transform(input_data['Preference'])
    input_data['red_wine_n'] = le_red_wine.transform(input_data['Red_Wine'])
    input_data['white_wine_n'] = le_white_wine.transform(input_data['White_Wine'])

    newdata = input_data.drop(["Preference", "Red_Wine", "White_Wine"], axis="columns")

    prediction = randomModel.predict(newdata)
    recommendation = prediction[0]

    recommendation_text = le_recommendation.inverse_transform([recommendation])[0]

    print(recommendation_text)

    
    return jsonify({'recommendation': recommendation_text})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [28/Mar/2024 13:53:20] "POST /random_forest HTTP/1.1" 200 -


Pinot-Noir
